<a href="https://colab.research.google.com/github/Vihith123/RAG_Application/blob/main/NutriChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Perform Google Colab installs (if running in Google Colab)
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    #!pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference


[INFO] Running in Google Colab, installing requirements.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 120.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash-attn
  Running setup.py clean for flash-attn
Failed to build flash-attn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (flash-attn)


In [ ]:
!pip uninstall -y torch torchvision torchaudio transformers sentence-transformers

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

!pip install -U transformers sentence-transformers


Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: transformers 4.57.2
Uninstalling transformers-4.57.2:
  Successfully uninstalled transformers-4.57.2
Found existing installation: sentence-transformers 5.1.2
Uninstalling sentence-transformers-5.1.2:
  Successfully uninstalled sentence-transformers-5.1.2
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 136.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 94.6 MB/s eta 0:00:00
     ━━━━

In [ ]:
# Download PDF file
import os
import requests

# Get PDF document
pdf_path = "human-nutrition-text.pdf"

# Download PDF if it doesn't already exist
if not os.path.exists(pdf_path):
    print("File doesn't exist, downloading...")

    # The URL of the PDF you want to download
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    # The local filename to save the downloaded file
    filename = pdf_path

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open a file in binary write mode and save the content to it
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"The file has been downloaded and saved as {filename}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
else:
    print(f"File {pdf_path} exists.")


File doesn't exist, downloading...
The file has been downloaded and saved as human-nutrition-text.pdf


In [ ]:
import fitz # This is the PyMuPdf
from tqdm.auto import tqdm # This Helps in faster iteration
def text_formatter(text: str) ->str:
  #Performs Minor operation on the text
  cleaned_text = text.replace("\n"," ").strip()
  return cleaned_text

# Now we will open the pdf
def open_and_read_pdf(pdf_path: str) ->list:
  doc = fitz.open(pdf_path) # opening a document
  pages_texts = [] # this is where we store the text contained in the certain number of page
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text)
    pages_texts.append({"page_number": page_number + 1,
                        "page_char_count": len(text),
                        "page_word_count": len(text.split(" ")),
                        "page_sentence_count": len(text.split(". ")),
                        "Page_token_count": len(text)/4,
                        "text": text
                        })
  return pages_texts # Moved return statement outside the loop

pages_texts = open_and_read_pdf(pdf_path = pdf_path)
pages_texts[:3]

0it [00:00, ?it/s]

[{'page_number': 1,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count': 1,
  'Page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': 2,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count': 1,
  'Page_token_count': 0.0,
  'text': ''},
 {'page_number': 3,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count': 1,
  'Page_token_count': 80.0,
  'text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'}]

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count,Page_token_count,text
0,1,29,4,1,7.25,Human Nutrition: 2020 Edition
1,2,0,1,1,0.00,
2,3,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,4,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,5,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [ ]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count,Page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,604.50,1148.00,198.30,9.97,287.00
std,348.86,560.38,95.76,6.19,140.10
min,1.00,0.00,1.00,1.00,0.00
25%,302.75,762.00,134.00,4.00,190.50
50%,604.50,1231.50,214.50,10.00,307.88
75%,906.25,1603.50,271.00,14.00,400.88
max,1208.00,2308.00,429.00,32.00,577.00


In [ ]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer") # It breaks Sentences in a paragraph into Single Sentences
doc = nlp("This is Vihith. I am a chess Player")
assert len(list(doc.sents))==2
list(doc.sents)

[This is Vihith., I am a chess Player]

In [ ]:
for item in tqdm(pages_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])


  0%|          | 0/1208 [00:00<?, ?it/s]

In [ ]:
import random
random.sample(pages_texts,k=1)

[{'page_number': 1151,
  'page_char_count': 1451,
  'page_word_count': 255,
  'page_sentence_count': 15,
  'Page_token_count': 362.75,
  'text': 'Supplementation may also be helpful to a limited degree. Vitamin  D and antioxidants have been linked to lowering the risk of some  cancers (however taking an iron supplement may promote others),  but, obtaining vital nutrients from food first is the best way to  help prevent or manage cancer. In addition, regular and vigorous  exercise can lower the risk of breast and colon cancers, among  others. Also, wear sunblock, stay in the shade, and avoid the midday  sun to protect yourself from skin cancer, which is one of the most  common kinds of cancer.8  Diabetes  What Is Diabetes?  Diabetes is one of the top three diseases in America. It affects  millions of people and causes tens of thousands of deaths each year.  Diabetes is a metabolic disease of insulin deficiency and glucose  over-sufficiency.  Like  other  diseases,  genetics,  nutrition,

In [ ]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])


  0%|          | 0/1208 [00:00<?, ?it/s]

In [ ]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = " ".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)  # ".A" -> ". A" for any full-stop/capital letter combo
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) // 4  # 1 token = ~4 characters

        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)



  0%|          | 0/1208 [00:00<?, ?it/s]

1843

In [ ]:
import random
random.sample(pages_and_chunks, k =1)

[{'page_number': 616,
  'sentence_chunk': 'Dietary Sources Biotin can be found in foods such as eggs, fish, meat, seeds, nuts and certain vegetables. For the pantothenic acid content of various foods, see Table 9.22 Biotin Content of Various Foods”. Table 9.22 Biotin Content of Various Foods Food Serving Biotin (mcg) Percent Daily Value* Eggs 1 large 10 33.3 Salmon, canned 3 oz. 5 16.6 Pork chop 3 oz. 3.8 12.6 Sunflower seeds ¼ c. 2.6 8.6 Sweet potato ½ c. 2.4 8 Almonds ¼ c. 1.5 5 Tuna, canned 3 oz. 0.6 2 Broccoli ½ c. 0.4 1.3 Banana ½ c. 0.2 0.6 * Current AI used to determine Percent Daily Value Fact Sheet for Health Professionals: Biotin. National Institute of Health, Office of Dietary Supplements. https://ods.od.nih.gov/ factsheets/Biotin-HealthProfessional/. Updated October 3, 2017. Accessed November 10, 2017. Vitamin B6 (Pyridoxine) Vitamin B6 is the coenzyme involved in a wide variety of functions in the body.',
  'chunk_char_count': 886,
  'chunk_word_count': 149,
  'chunk_token

In [ ]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.00,1843.00,1843.00,1843.00
mean,625.38,735.14,113.03,183.41
std,347.79,447.64,71.27,111.91
min,1.00,12.00,3.00,3.00
25%,322.50,315.00,45.00,78.00
50%,628.00,747.00,114.00,186.00
75%,932.00,1119.00,174.00,279.00
max,1208.00,1832.00,298.00,458.00


In [ ]:
min_chunk_char_count = 30
pages_and_chunks_over_min_length = [
    chunk for chunk in pages_and_chunks if chunk["chunk_char_count"] >= min_chunk_char_count
]

print(f"Number of chunks before filtering: {len(pages_and_chunks)}")
print(f"Number of chunks after filtering (min char count >= {min_chunk_char_count}): {len(pages_and_chunks_over_min_length)}")


Number of chunks before filtering: 1843
Number of chunks after filtering (min char count >= 30): 1822


In [ ]:
pages_and_chunks_over_min_length[:2]

[{'page_number': 3,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77},
 {'page_number': 4,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52}]

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device="cpu") # Choose the device to load the model to (note: GPU will often be *
# Create a list of sentences to turn into numbers
sentences = [
    "The Sentences Transformers library provides an easy and open-source way to create embeddings.",
    "Sentences can be embedded one by one or as a list of strings.",
    "Embeddings are one of the most powerful concepts in machine learning!",
    "Learn to use embeddings well and you'll be well on your way to being an AI engineer."
]

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(sentences)
embeddings_dict = dict(zip(sentences, embeddings))

# See the embeddings
for sentence, embedding in embeddings_dict.items():
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Sentence: The Sentences Transformers library provides an easy and open-source way to create embeddings.
Embedding: [-2.07981113e-02  3.03165037e-02 -2.01217793e-02  6.86483830e-02
 -2.55255643e-02 -8.47689249e-03 -2.07147663e-04 -6.32377118e-02
  2.81606149e-02 -3.33353840e-02  3.02634649e-02  5.30720763e-02
 -5.03526479e-02  2.62288153e-02  3.33314016e-02 -4.51578461e-02
  3.63043845e-02 -1.37116178e-03 -1.20171569e-02  1.14946812e-02
  5.04510887e-02  4.70857173e-02  2.11912915e-02  5.14607765e-02
 -2.03746371e-02 -3.58889103e-02 -6.67888962e-04 -2.94393133e-02
  4.95858490e-02 -1.05639603e-02 -1.52013749e-02 -1.31754903e-03
  4.48196754e-02  1.56023391e-02  8.60379942e-07 -1.21397164e-03
 -2.37978995e-02 -9.09372000e-04  7.34479493e-03 -2.53932923e-03
  5.23370057e-02 -4.68043573e-02  1.66214500e-02  4.71579209e-02
 -4.15599570e-02  9.01942665e-04  3.60278897e-02  3.42214778e-02
  9.68227163e-02  5.94828576e-02 -1.64984670e-02 -3.51250097e-02
  5.92516316e-03 -7.07960629e-04 -2.4103

In [ ]:
embedding_model.to("cuda")
for item in tqdm(pages_and_chunks_over_min_length):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])
#

  0%|          | 0/1822 [00:00<?, ?it/s]

In [ ]:
text_chunk_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_length)
embeddings_df_save_path = "text_chunk_and_embeddings_df.csv"

text_chunk_and_embeddings_df.to_csv(embeddings_df_save_path, index = False)

In [ ]:
text_chunk_and_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_chunk_and_embeddings_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,3,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77,[ 6.74242675e-02 9.02281404e-02 -5.09548886e-...
1,4,Human Nutrition: 2020 Edition by University of...,210,30,52,[ 5.52156419e-02 5.92139773e-02 -1.66167244e-...
2,5,Contents Preface University of Hawai‘i at Māno...,766,114,191,[ 2.79801842e-02 3.39813754e-02 -2.06426680e-...
3,6,Lifestyles and Nutrition University of Hawai‘i...,942,143,235,[ 6.82566911e-02 3.81275006e-02 -8.46854132e-...
4,7,The Cardiovascular System University of Hawai‘...,998,152,249,[ 3.30264494e-02 -8.49763490e-03 9.57159605e-...
